In [6]:
import pandas as pd
import requests

# Read the CSV files into pandas DataFrames
location_costs = pd.read_csv('https://raw.githubusercontent.com/kflemming30/6505-Project/main/cost_of_living_us.csv')

airbnb_property_reviews = pd.read_csv('https://raw.githubusercontent.com/kflemming30/6505-Project/main/Airbnb_Open_Data.csv',
    dtype=str) 

temperatures = pd.read_csv("C:\\Users\\12039\\Documents\\6505-Project\\city_temperature.csv",dtype=str)

Seattle_listings = pd.read_csv("Seattle_listings.csv",dtype=str)


In [22]:
location_costs.head()

,case_id,state,isMetro,areaname,county,family_member_count,housing_cost,food_cost,transportation_cost,healthcare_cost,other_necessities_cost,childcare_cost,taxes,total_cost,median_family_income
0,1,AL,True,"Montgomery, AL MSA",Autauga County,1p0c,8505.72876,3454.91712,10829.16876,5737.47984,4333.81344,0.0000,6392.94504,39254.0532,73010.414062
1,1,AL,True,"Montgomery, AL MSA",Autauga County,1p1c,12067.50240,5091.70788,11588.19288,8659.55640,6217.45896,6147.8298,7422.07836,57194.3256,73010.414062
2,1,AL,True,"Montgomery, AL MSA",Autauga County,1p2c,12067.50240,7460.20308,12361.77720,11581.63260,7075.65816,15824.6940,9769.56228,76141.0308,73010.414062
3,1,AL,True,"Montgomery, AL MSA",Autauga County,1p3c,15257.15040,9952.23924,13452.18600,14503.70760,9134.35620,18802.1892,13101.70320,94203.5328,73010.414062
4,1,AL,True,"Montgomery, AL MSA",Autauga County,1p4c,15257.15040,12182.21400,13744.59840,17425.78560,9942.36396,18802.1892,13469.21880,100823.5200,73010.414062


In [11]:
Seattle_listings.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6606,https://www.airbnb.com/rooms/6606,20230918050357,2023-09-18,city scrape,Guesthouse in Seattle · ★4.60 · 1 bedroom · 1 ...,"This tiny cottage is only 15x10, but it has ev...","A peaceful yet highly accessible neighborhood,...",https://a0.muscache.com/pictures/45742/21116d7...,14942,...,4.76,4.88,4.57,str-opli-19-002622,f,2,2,0,0,0.93
1,9419,https://www.airbnb.com/rooms/9419,20230918050357,2023-09-18,city scrape,Rental unit in Seattle · ★4.72 · 1 bedroom · 1...,"Keeping you safe is our priority, we are adher...","Lots of restaurants (see our guide book) bars,...",https://a0.muscache.com/pictures/56645186/e5fb...,30559,...,4.89,4.66,4.65,STR-OPLI-19-003039,f,9,0,9,0,1.21
2,9531,https://www.airbnb.com/rooms/9531,20230918050357,2023-09-18,city scrape,Home in Seattle · ★4.96 · 2 bedrooms · 3 beds ...,The Sweet Orange is a delightful and spacious ...,The neighborhood is awesome! Just far enough ...,https://a0.muscache.com/pictures/30470355/052c...,31481,...,4.96,4.97,4.91,STR-OPLI-19-002182,f,2,2,0,0,0.54
3,9534,https://www.airbnb.com/rooms/9534,20230918050357,2023-09-18,city scrape,Guest suite in Seattle · ★4.99 · 2 bedrooms · ...,Welcome to my delicious Tangerine Dream! A co...,The neighborhood is the best of two worlds...w...,https://a0.muscache.com/pictures/30476721/0751...,31481,...,4.99,4.96,4.95,STR-OPLI-19-002182,f,2,2,0,0,0.53
4,9596,https://www.airbnb.com/rooms/9596,20230918050357,2023-09-18,previous scrape,Rental unit in Seattle · ★4.56 · 1 bedroom · 4...,"We are in a great neighborhood, quiet, full of...","if you arrive early for check in at 3, I reco...",https://a0.muscache.com/pictures/665252/102d18...,14942,...,4.66,4.91,4.56,STR -OPLI-19-002622,f,2,2,0,0,0.65


In [ ]:
#retrieve data from url directly instead of saving elsewhere.  Not sure this is best

# import requests
# import gzip
# import shutil

# url = 'http://data.insideairbnb.com/united-states/wa/seattle/2023-09-18/data/reviews.csv.gz'
# Seattle_reviews_file_path = 'reviews.csv.gz'

# # Fetch the file from the URL
# Seattle_reviews_response = requests.get(url, stream=True)

# # Save the file to disk
# with open(Seattle_reviews_file_path, 'wb') as file:
#     shutil.copyfileobj(Seattle_reviews_response.raw, file)

# # Read the CSV file into a pandas DataFrame
# with gzip.open(Seattle_reviews_file_path, 'rb') as gz_file:
#     Seattle_reviews = pd.read_csv(gz_file)

# # Display a preview of the DataFrame
# Seattle_reviews.head()

In [23]:
# create dataframes
Seattle_reviews = pd.read_csv("Seattle_reviews.csv")
LA_reviews = pd.read_csv("LA_reviews.csv")

#convert date so that we can filter on 2023 and use that file of review details
Seattle_reviews['date'] = pd.to_datetime(Seattle_reviews['date'])
Seattle_reviews['date'].dt.year.unique()
LA_reviews['date'] = pd.to_datetime(LA_reviews['date'])

In [24]:
filter_year = 2023

#create subset of review details for 2023 only
Seattle_reviews_2023 = Seattle_reviews[Seattle_reviews['date'].dt.year == filter_year]
LA_reviews_2023 =LA_reviews[LA_reviews['date'].dt.year == filter_year]

# Save Seattle reviews subset for 2023 to a CSV file
Seattle_reviews_2023.to_csv('Seattle_reviews_2023.csv', index=False)

# Save LA reviews subset for 2023 to a CSV file
LA_reviews_2023.to_csv('LA_reviews_2023.csv', index=False)